In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

First we will load and clean our data!

In [61]:
f = np.load("moonboard.npz")
X_train, y_train = f['x_train'], f['y_train']
X_test,  y_test  = f['x_test'],  f['y_test']

Here we can take a look at our labels (the grades encompassed in our dataset):

In [62]:
print(set(y_train))#grade labels!
np.where(y_train == 0)

{0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}


(array([4099], dtype=int64),)

As we can see, there is only one climb labeled with a grade of 0 - the lowest possible grade for a Moonboard climb is V4, so we will remove it.
We need to remove the climb itself as well as the corresponding label.

In [63]:
X_train = np.delete(X_train, 4099, axis=0)
y_train = np.delete(y_train, 4099)

In [64]:
y_test = y_test - 4 # normalizing to 0-12 for sparse categorical cross-entropy
y_train = y_train - 4

In [65]:
print(X_train.shape)
print(y_train.shape)

(9523, 18, 11)
(9523,)


In [66]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(18,11,1)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(13, activation='softmax')

    ]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 16, 9, 32)         320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 4, 32)         0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 8, 4, 32)         128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 2, 64)          18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 1, 64)         0         
 2D)                                                             
                                                      

In [67]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
149/149 [==============================] - 1s 5ms/step - loss: 1.9641 - accuracy: 0.3076
Epoch 2/100
149/149 [==============================] - 1s 5ms/step - loss: 1.7709 - accuracy: 0.3476
Epoch 3/100
149/149 [==============================] - 1s 5ms/step - loss: 1.6934 - accuracy: 0.3726
Epoch 4/100
149/149 [==============================] - 1s 5ms/step - loss: 1.6463 - accuracy: 0.3884
Epoch 5/100
149/149 [==============================] - 1s 5ms/step - loss: 1.5950 - accuracy: 0.3982
Epoch 6/100
149/149 [==============================] - 1s 5ms/step - loss: 1.5545 - accuracy: 0.4100
Epoch 7/100
149/149 [==============================] - 1s 5ms/step - loss: 1.5267 - accuracy: 0.4253
Epoch 8/100
149/149 [==============================] - 1s 5ms/step - loss: 1.4755 - accuracy: 0.4498
Epoch 9/100
149/149 [==============================] - 1s 5ms/step - loss: 1.4558 - accuracy: 0.4563
Epoch 10/100
149/149 [==============================] - 1s 5ms/step - loss: 1.4111 - accura

In [68]:
model.evaluate(X_test, y_test)

75/75 [==============================] - 0s 1ms/step - loss: 1.9305 - accuracy: 0.6805


[1.9305113554000854, 0.6805205941200256]

In [69]:
y_pred = model.predict(X_test)
y_pred.shape == y_test.shape

75/75 [==============================] - 0s 1ms/step


False

In [74]:
max_probability_grades = []
for pred in y_pred:
    max_probability_grades.append(np.argmax(pred))

In [76]:
preds_within_one = 0

for i in range(len(y_test)):
    if np.abs(y_test[i] - max_probability_grades[i]) <= 1:
        preds_within_one += 1


In [77]:
preds_within_one / len(y_test)

0.8073047858942065

In [56]:
nimbus = np.array([
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,1],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

nimbus = nimbus.reshape((1, 18, 11, 1))

easy_prob = np.array([
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

easy_prob = easy_prob.reshape((1,18,11,1))

whispering_aspens = np.array([
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

whispering_aspens = whispering_aspens.reshape((1,18,11,1))

nimbus_pred = model.predict(nimbus)
ep_output = model.predict(easy_prob)
aspens_pred = model.predict(whispering_aspens)

1/1 [==============================] - 0s 11ms/step


In [57]:
nimbus_pred

array([[6.87273499e-03, 1.29098790e-02, 8.40305448e-01, 1.39785901e-01,
        1.24971193e-04, 1.01280406e-07, 8.62599393e-07, 2.63170215e-11,
        4.56410916e-11, 9.80617172e-13, 2.54237373e-19, 1.28798994e-11,
        8.31755279e-16]], dtype=float32)

In [58]:
np.argmax(ep_output)

0

In [59]:
np.argmax(nimbus_pred)

2

In [60]:
np.argmax(aspens_pred)

7